In [1]:
import pandas as pd
import warnings as wn

In [2]:
alarm_dataset=pd.read_csv('Combined_Alarm_Dataset.csv')
alarm_dataset

,Date,Device,Details,Deactivedate,Duration
0,2018-11-16 21:59:19,Plant,DelREMO is not communicating.,2018-11-17 04:20:29,06:21:10
1,2018-11-16 21:59:19,Plant,DelREMO is not communicating.,2018-11-17 04:05:21,06:06:02
2,2018-11-17 21:58:58,Plant,DelREMO is not communicating.,2018-11-18 04:05:18,06:06:20
3,2018-11-17 21:58:58,Plant,DelREMO is not communicating.,2018-11-18 04:20:26,06:21:28
4,2018-11-18 21:58:10,Plant,DelREMO is not communicating.,2018-11-19 04:05:17,06:07:07
...,...,...,...,...,...
2644,2021-07-21 20:20:04,Plant,DelREMO is not communicating.,2021-07-21 20:20:04,00:00:00
2645,2021-07-21 20:25:06,Plant,DelREMO is not communicating.,2021-07-21 20:30:08,00:05:02
2646,2021-07-21 20:30:08,Plant,DelREMO is not communicating.,2021-07-21 20:35:10,00:05:02
2647,2021-07-23 07:16:21,Plant,DelREMO is not communicating.,2021-07-23 10:20:59,03:04:38


In [3]:
def processSiteEvents(data):

    ''' function takes site events data as input, and create new variables and output a dataframe '''
    
    wn.filterwarnings('ignore')
    #df = pd.read_csv(fileName)
    df=data
    
    df['Date'] = pd.to_datetime(df['Date'])
    
    df['Deactivedate'] = pd.to_datetime(df['Deactivedate'])
    
    # derive new variables
    df['EventDate'] = df['Date'].dt.date
    
    df['EventDuration'] = (pd.to_datetime(df['Deactivedate']) - pd.to_datetime(df['Date']))
    
    df['EventDurationSecs'] = df['EventDuration'].dt.total_seconds()
    df['EventDurationMins'] = df['EventDuration'].dt.total_seconds()/60
    df['EventDurationHours'] = df['EventDuration'].dt.total_seconds()/60/60
    
    df['EventDay'] = df['Date'].dt.day
    df['EventMon'] = df['Date'].dt.month
    df['EventYear'] = df['Date'].dt.year
    
    df['EventWeekday'] = df['Date'].dt.day_name()
    df['EventMonth'] = df['Date'].dt.month_name()

    df['EventHour'] = df['Date'].dt.hour
    
    bins = [0, 4, 8, 12, 16, 20, 24]
    labels = ['Late Night', 'Early Morning', 'Morning', 'Afternoon', 'Evening', 'Night']

    df['EventPeriod'] = pd.cut(df['EventHour'], bins = bins, labels = labels, include_lowest = True)
    
#     if (processed.Details=='E09: No Grid') | (processed.Details=='Phase1 Grid Voltage High') | (processed.Details=='Phase1 Grid Voltage Low') | (processed.Details=='Grid Frequency out of range'):
#         processed['EventCategory']='Grid'
#     elif (processed['Details']=='Hardware problem') | (processed['Details']=='HW Realy Short') | (processed['Details']=='HW Relay open'):
#         processed['EventCategory']='Maintenance'
#     else:
#         processed['EventCategory']=None

    Details_dict ={'Phase1 Grid Voltage Low' : 'Grid', 'Phase1 Grid Voltage Low' : 'Grid', 'E09: No Grid' : 'Grid', 'Phase1 Grid Voltage High' : 'Grid',
                  'Grid Frequency out of range' : 'Grid', 'Hardware problem' : 'Maintenance', 'HW Realy Short' : 'Maintenance',
                  'HW Relay open' : 'Maintenance', 'DelREMO is not communicating.' : 'DelRemo'}

    df['EventCategory'] = df['Details'].map(Details_dict)
    
    dfDel = df[(df.EventCategory == 'DelRemo') & (df.EventDurationHours >= 0) & (df.EventDurationHours <= 24) & (df.EventPeriod!='Late Night') & (df.EventPeriod!='Night')]
    dfGrid = df[(df.EventCategory == 'Grid') & (df.EventDurationHours >= 0) & (df.EventDurationHours <= 24)]
    dfPV = df[(df.EventCategory == 'Maintenance') & (df.EventDurationHours <= 96)]

    df = pd.concat([dfDel,dfGrid, dfPV])
    df=df.sort_values(by='Date')
    
    return df

In [4]:
processed=processSiteEvents(alarm_dataset)
processed.head(10)

,Date,Device,Details,Deactivedate,Duration,EventDate,EventDuration,EventDurationSecs,EventDurationMins,EventDurationHours,EventDay,EventMon,EventYear,EventWeekday,EventMonth,EventHour,EventPeriod,EventCategory
5,2018-11-19 16:22:57,1,Phase1 Grid Voltage Low,2018-11-19 16:23:04,00:00:07,2018-11-19,0 days 00:00:07,7.0,0.116667,0.001944,19,11,2018,Monday,November,16,Afternoon,Grid
7,2018-11-20 07:43:16,1,Phase1 Grid Voltage Low,2018-11-20 07:43:23,00:00:07,2018-11-20,0 days 00:00:07,7.0,0.116667,0.001944,20,11,2018,Tuesday,November,7,Early Morning,Grid
11,2018-11-22 08:07:51,Plant,DelREMO is not communicating.,2018-11-22 08:22:59,00:15:08,2018-11-22,0 days 00:15:08,908.0,15.133333,0.252222,22,11,2018,Thursday,November,8,Early Morning,DelRemo
12,2018-11-22 10:16:30,1,Phase1 Grid Voltage Low,2018-11-22 10:16:38,00:00:08,2018-11-22,0 days 00:00:08,8.0,0.133333,0.002222,22,11,2018,Thursday,November,10,Morning,Grid
13,2018-11-22 11:56:50,1,Phase1 Grid Voltage Low,2018-11-22 11:56:58,00:00:08,2018-11-22,0 days 00:00:08,8.0,0.133333,0.002222,22,11,2018,Thursday,November,11,Morning,Grid
14,2018-11-22 15:36:17,1,Phase1 Grid Voltage Low,2018-11-22 15:36:24,00:00:07,2018-11-22,0 days 00:00:07,7.0,0.116667,0.001944,22,11,2018,Thursday,November,15,Afternoon,Grid
16,2018-11-23 11:36:51,1,Phase1 Grid Voltage Low,2018-11-23 11:36:59,00:00:08,2018-11-23,0 days 00:00:08,8.0,0.133333,0.002222,23,11,2018,Friday,November,11,Morning,Grid
19,2018-11-24 12:20:00,1,Phase1 Grid Voltage Low,2018-11-24 12:20:08,00:00:08,2018-11-24,0 days 00:00:08,8.0,0.133333,0.002222,24,11,2018,Saturday,November,12,Morning,Grid
20,2018-11-24 15:19:46,1,Phase1 Grid Voltage Low,2018-11-24 15:19:53,00:00:07,2018-11-24,0 days 00:00:07,7.0,0.116667,0.001944,24,11,2018,Saturday,November,15,Afternoon,Grid
21,2018-11-24 15:21:25,1,HW Realy Short,2018-11-24 17:40:27,02:19:02,2018-11-24,0 days 02:19:02,8342.0,139.033333,2.317222,24,11,2018,Saturday,November,15,Afternoon,Maintenance


In [5]:
processed.shape

(1563, 18)

In [6]:
processed.to_csv('./processed_alarm_Dataset.csv',index=False)

In [7]:
processed.Details.unique()

array(['Phase1 Grid Voltage Low', 'DelREMO is not communicating.',
       'HW Realy Short', 'Phase1 Grid Voltage High', 'Hardware problem',
       'E09: No Grid', 'Grid Frequency out of range'], dtype=object)

In [8]:
def transformEvents(filename):
    
    eventsSummDF = processed.groupby(['EventDate', 'Details']).agg({'EventDurationHours':['sum']})
    eventsSummDF.columns = ['EventDurationHours']    
    eventsSummDF = eventsSummDF.reset_index()
    
    pivoted_alarm_dataset=pd.pivot_table(eventsSummDF, index=['EventDate'],columns='Details',
                                         values='EventDurationHours')
    #pivoted_alarm_dataset=pivoted_alarm_dataset.drop(columns=['DelREMO is not communicating.'])
    pivoted_alarm_dataset=pivoted_alarm_dataset.reset_index()
    pivoted_alarm_dataset=pivoted_alarm_dataset.fillna(0)
    
    pivoted_alarm_dataset['GridDurationHours']=(pivoted_alarm_dataset['E09: No Grid']+
                                              pivoted_alarm_dataset['Phase1 Grid Voltage High']+
                                              pivoted_alarm_dataset['Phase1 Grid Voltage Low']+
                                              pivoted_alarm_dataset['Grid Frequency out of range'])                      
    pivoted_alarm_dataset['MaintDurationHours']=(pivoted_alarm_dataset['Hardware problem']+
                                                 pivoted_alarm_dataset['HW Realy Short']) #+pivoted_alarm_dataset['HW Relay open'])
    pivoted_alarm_dataset=pivoted_alarm_dataset.rename(columns={'DelREMO is not communicating.':'DelREMO'})
    return pivoted_alarm_dataset

In [9]:
pivoted_alarm_dataset=transformEvents(processed)
pivoted_alarm_dataset

Details,EventDate,DelREMO,E09: No Grid,Grid Frequency out of range,HW Realy Short,Hardware problem,Phase1 Grid Voltage High,Phase1 Grid Voltage Low,GridDurationHours,MaintDurationHours
0,2018-11-19,0.000000,0.0,0.0,0.000000,0.0,0.0,0.001944,0.001944,0.000000
1,2018-11-20,0.000000,0.0,0.0,0.000000,0.0,0.0,0.001944,0.001944,0.000000
2,2018-11-22,0.252222,0.0,0.0,0.000000,0.0,0.0,0.006389,0.006389,0.000000
3,2018-11-23,0.000000,0.0,0.0,0.000000,0.0,0.0,0.002222,0.002222,0.000000
4,2018-11-24,0.000000,0.0,0.0,2.317222,0.0,0.0,0.006111,0.006111,2.317222
...,...,...,...,...,...,...,...,...,...,...
423,2021-07-17,0.251389,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
424,2021-07-18,0.167500,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
425,2021-07-20,0.083889,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
426,2021-07-21,0.502778,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000


In [10]:
pivoted_alarm_dataset.to_csv('./pivoted_alarm_Dataset.csv',index=False)